In [63]:
import pandas as pd
import numpy as np
from data import CleanedData
from models import nn
import sklearn.metrics
import torch
import matplotlib.pyplot as plt

# For Adaboost
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import pickle

# For Adaboost Tuning
from sklearn import preprocessing
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

# For KNN
from sklearn.neighbors import KNeighborsClassifier
import sklearn
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import pickle
import matplotlib.pyplot as plts
import time
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# For NN Tuning
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import recall_score
import pandas as pd
import numpy as np
import torch.nn
import optuna
import torch
import tqdm

## Neural Network Tuning

In [76]:
data = CleanedData(loc='./data/joined.csv', test_ratio=0.2, impute_data=True, convert_non_numerical=True, normalize_data=True, drop_columns=['source', 'Last_Update','province','country','latitude', 'longitude'])

In [77]:

def objective(trial):
    global data
    i = 0
    first_layer = trial.suggest_int('first_layer', 20, 250)
    second_layer = trial.suggest_int('second_layer', 1, 200)
    activation_1 = trial.suggest_categorical('activation1', ['relu',])
    activation_2 = trial.suggest_categorical('activation2', ['sigmoid',])
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    acc = 0
    old_recall = np.zeros(4)
    old_overal_recall = np.zeros(4)
    for epoch, (X_train, y_train, X_val, y_val) in enumerate(data.kfold_data):
        train_dataloader = nn.load_data(X_train, y_train)
        val_dataloader = nn.load_data(X_val, y_val, len(y_val), train=False)
        optimizer.zero_grad()
        model = nn.network(first_layer, second_layer, activation_1, activation_2)
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        for i in range(20):
            for batch_idx, (X, y) in enumerate(train_dataloader):
                model.train()
                y_hat = model(X)
                loss = criterion(y_hat, y.long())
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            for val_x, val_y in val_dataloader:
                model.eval()
                y_h = model(val_x)
                overal_recall = recall_score(val_y.numpy(), torch.argmax(torch.softmax(y_h,dim=1), dim=1).numpy(), average=None,zero_division=0)
                old_overal_recall = np.sum([overal_recall, old_overal_recall], axis=0)
                i+=1
            trial.report(old_overal_recall[0]/i, epoch)
            trial.set_user_attr('overall recall', old_overal_recall/i)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
    print(old_overal_recall/i)
    return old_overal_recall[0]/i

In [78]:
study = optuna.create_study(sampler=optuna.samplers.TPESampler(),direction="maximize")
study.optimize(objective, n_trials=500)

[I 2020-12-15 00:15:23,396] A new study created in memory with name: no-name-3d1851c0-dd7f-4aef-be16-b8f9d6f57a0a
[I 2020-12-15 00:19:56,452] Trial 0 finished with value: 0.0 and parameters: {'first_layer': 186, 'second_layer': 52, 'activation1': 'relu', 'activation2': 'sigmoid', 'lr': 0.08397629924649405}. Best is trial 0 with value: 0.0.


[0. 0. 4. 0.]


[I 2020-12-15 00:24:26,309] Trial 1 finished with value: 0.0 and parameters: {'first_layer': 237, 'second_layer': 48, 'activation1': 'relu', 'activation2': 'sigmoid', 'lr': 1.0267366558207516e-05}. Best is trial 0 with value: 0.0.


[0.         2.32177456 3.99714225 0.        ]


[I 2020-12-15 00:28:51,861] Trial 2 finished with value: 0.0 and parameters: {'first_layer': 141, 'second_layer': 48, 'activation1': 'relu', 'activation2': 'sigmoid', 'lr': 0.02416859009435577}. Best is trial 0 with value: 0.0.


...

[I 2020-12-15 11:19:38,345] Trial 149 finished with value: 0.09533502334889926 and parameters: {'first_layer': 25, 'second_layer': 189, 'activation1': 'relu', 'activation2': 'sigmoid', 'lr': 0.0006731664308089185}. Best is trial 73 with value: 0.12482632866355348.


[0.09533502 1.27657606 3.97535846 0.82243943]


[I 2020-12-15 11:24:03,066] Trial 150 finished with value: 0.08547191460973982 and parameters: {'first_layer': 24, 'second_layer': 191, 'activation1': 'relu', 'activation2': 'sigmoid', 'lr': 0.0006806940944017219}. Best is trial 73 with value: 0.12482632866355348.


[0.08547191 1.60962859 3.97353118 1.1310676 ]


KeyboardInterrupt: 

In [159]:
df = study.trials_dataframe().sort_values(by='value', ascending=False)

,number,value,datetime_start,datetime_complete,duration,params_activation1,params_activation2,params_first_layer,params_lr,params_second_layer,state,deceased_recall,hospitalized_recall,nonhospitalized_recall,recovered_recall,mean_recall
73,73,0.124826,2020-12-15 05:48:35.712081,2020-12-15 05:52:49.920706,0 days 00:04:14.208625,relu,sigmoid,48,0.000848,182,COMPLETE,0.031207,0.270311,0.995183,0.495260,0.447990
149,149,0.095335,2020-12-15 11:15:31.265021,2020-12-15 11:19:38.345234,0 days 00:04:07.080213,relu,sigmoid,25,0.000673,189,COMPLETE,0.023834,0.319144,0.993840,0.205610,0.385607
99,99,0.094134,2020-12-15 07:39:40.532166,2020-12-15 07:43:48.502671,0 days 00:04:07.970505,relu,sigmoid,31,0.002349,168,COMPLETE,0.023534,0.364526,0.995485,0.146473,0.382504
80,80,0.090545,2020-12-15 06:17:56.396586,2020-12-15 06:21:56.203684,0 days 00:03:59.807098,relu,sigmoid,37,0.000522,171,COMPLETE,0.022636,0.370768,0.994534,0.258851,0.411697
116,116,0.086675,2020-12-15 08:49:07.835924,2020-12-15 08:52:55.089233,0 days 00:03:47.253309,relu,sigmoid,20,0.000411,169,COMPLETE,0.021669,0.361870,0.994109,0.417477,0.448781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,35,0.000000,2020-12-15 03:02:28.871451,2020-12-15 03:07:01.166841,0 days 00:04:32.295390,relu,sigmoid,237,0.003161,33,COMPLETE,0.000000,0.000000,1.000000,0.000000,0.250000
34,34,0.000000,2020-12-15 02:57:43.476632,2020-12-15 03:02:28.870437,0 days 00:04:45.393805,relu,sigmoid,189,0.002062,77,COMPLETE,0.000000,0.266777,0.999213,0.002574,0.317141
33,33,0.000000,2020-12-15 02:52:41.210965,2020-12-15 02:57:43.475572,0 days 00:05:02.264607,relu,sigmoid,163,0.000184,130,COMPLETE,0.000000,0.245704,0.999821,0.363034,0.402140
32,32,0.000000,2020-12-15 02:48:45.657919,2020-12-15 02:52:41.210102,0 days 00:03:55.552183,relu,sigmoid,106,0.012740,49,COMPLETE,0.000000,0.059472,0.999718,0.000000,0.264797


In [160]:
df.to_csv('/results/nn_trial.csv')

## Adaboost Tuning

In [2]:
data = CleanedData(loc='./data/joined.csv.gz', test_ratio=0.2, impute_data=True, convert_non_numerical=True, normalize_data=True, drop_columns=['source', 'Last_Update','province','country','latitude', 'longitude'])

D:\Users\gerla\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [3]:
X_test, y_test = data.test_data

In [48]:
X = data.data.drop(columns=['outcome', 'sex', 'date_confirmation', 'Combined_Key']).to_numpy()
y = data.data['outcome'].to_numpy()

In [13]:
X

array([[ 43.02504491,  22.694884  ,  71.590923  , ...,   0.        ,
        169.        ,   2.        ],
       [ 94.        , -11.766533  , -76.604498  , ...,   0.        ,
        238.        ,   0.        ],
       [ 40.59487903,  22.694884  ,  71.590923  , ...,   0.        ,
        169.        ,   2.        ],
       ...,
       [ 39.        , -11.00523373, -76.15759133, ...,   0.        ,
        352.        ,   0.        ],
       [ 66.        ,   6.02517268, -74.6520927 , ...,   0.        ,
        102.        ,   0.        ],
       [ 27.77693403,  20.06780113,  79.02780697, ...,   0.        ,
        195.        ,   2.        ]])

In [49]:
y

array([1, 2, 1, ..., 2, 2, 1])

In [37]:
def deceased_recall(y_true, y_pred):
    scores = recall_score(y_true, y_pred, average=None)
    return scores[0]

In [99]:
def overall_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average = 'micro')

In [100]:
param_grid = [{'n_estimators': [10, 50, 100],
'learning_rate':[1,2,5,10]}]

scoring = {'Accuracy': make_scorer(accuracy_score), 'Overall_Recall': make_scorer(overall_recall), 'deceased_Recall': make_scorer(deceased_recall)}

grid = GridSearchCV(AdaBoostClassifier(), param_grid=param_grid, scoring=scoring, refit='Overall_Recall', n_jobs = -1, cv = 4)

In [101]:
grid.fit(X,y)

GridSearchCV(cv=4, estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid=[{'learning_rate': [1, 2, 5, 10],
                          'n_estimators': [10, 50, 100]}],
             refit='Overall_Recall',
             scoring={'Accuracy': make_scorer(accuracy_score),
                      'Overall_Recall': make_scorer(overall_recall),
                      'deceased_Recall': make_scorer(deceased_recall)})

In [108]:
results = pd.DataFrame(grid.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_n_estimators,params,split0_test_Accuracy,split1_test_Accuracy,split2_test_Accuracy,...,mean_test_Overall_Recall,std_test_Overall_Recall,rank_test_Overall_Recall,split0_test_deceased_Recall,split1_test_deceased_Recall,split2_test_deceased_Recall,split3_test_deceased_Recall,mean_test_deceased_Recall,std_test_deceased_Recall,rank_test_deceased_Recall
0,21.244448,0.233944,1.387744,0.026093,1,10,"{'learning_rate': 1, 'n_estimators': 10}",0.809528,0.811665,0.766408,...,0.788615,0.021995,2,0.006000,0.000000,0.000000,0.000000,0.001500,0.002598,11
1,105.294025,1.605846,6.334218,0.099496,1,50,"{'learning_rate': 1, 'n_estimators': 50}",0.807282,0.783747,0.721660,...,0.745508,0.053964,5,0.001333,0.008005,0.008000,0.326000,0.085835,0.138686,8
2,210.577762,4.673932,12.694044,0.103358,1,100,"{'learning_rate': 1, 'n_estimators': 100}",0.803959,0.733624,0.701702,...,0.704250,0.081888,6,0.020667,0.038025,0.034667,0.582000,0.168840,0.238627,6
3,21.124957,0.405476,1.359465,0.028504,2,10,"{'learning_rate': 2, 'n_estimators': 10}",0.812693,0.812706,0.812763,...,0.783235,0.051072,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12
4,104.859853,2.201875,6.700093,0.222943,2,50,"{'learning_rate': 2, 'n_estimators': 50}",0.812693,0.812706,0.812763,...,0.797158,0.026956,1,0.000000,0.000000,0.000000,0.081333,0.020333,0.035218,10
5,210.009666,4.477447,12.758247,0.264183,2,100,"{'learning_rate': 2, 'n_estimators': 100}",0.812693,0.812706,0.812763,...,0.781955,0.053287,4,0.000000,0.000000,0.000000,0.500667,0.125167,0.216795,7
6,18.740341,1.532921,1.372905,0.044796,5,10,"{'learning_rate': 5, 'n_estimators': 10}",0.172774,0.361038,0.340103,...,0.409146,0.216755,7,0.014000,0.000000,0.095333,0.018667,0.032000,0.037205,9
7,87.542291,5.480122,6.320621,0.079580,5,50,"{'learning_rate': 5, 'n_estimators': 50}",0.455202,0.277234,0.389817,...,0.352781,0.073577,8,0.056000,0.043362,0.038667,0.742667,0.220174,0.301728,5
8,114.280915,33.700312,11.083940,1.352745,5,100,"{'learning_rate': 5, 'n_estimators': 100}",0.043455,0.454014,0.410121,...,0.350156,0.179487,9,0.809333,0.006004,0.488000,0.272667,0.394001,0.294364,1
9,11.469687,1.132709,1.395216,0.036918,10,10,"{'learning_rate': 10, 'n_estimators': 10}",0.026978,0.007658,0.010765,...,0.124459,0.189499,10,0.000000,0.000000,1.000000,0.375333,0.343833,0.408653,2


In [111]:
results.to_csv("./results/adaboost_output.csv")

## K Nearest Neighbours (KNN) Tuning

In [14]:
X = data.data.drop(columns=['outcome', 'sex', 'date_confirmation', 'Combined_Key']).to_numpy()
y = data.data['outcome'].to_numpy()

In [15]:
def overall_recall(y_true, y_pred):
    score = recall_score(y_true, y_pred, average='macro')
    return score

def deceased_recall(y_true, y_pred):
    score = recall_score(y_true, y_pred, average=None)
    return score[0]

In [16]:
grid_params = {
    'n_neighbors': [38,40,42,44],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

## Overall_Recall
#gs = GridSearchCV(
#    KNeighborsClassifier(),
#    grid_params,
#    verbose = 1,
#    cv = 4,
#    n_jobs= -1,
#    scoring=make_scorer(overall_recall, greater_is_better=True)
#    )

# Accuracy
#gs = GridSearchCV(
#    KNeighborsClassifier(),
#    grid_params,
#    verbose = 1,
#    cv = 4,
#    n_jobs= -1,
#    scoring='accuracy'
#    )

scoring = make_scorer(deceased_recall)

# Deceased Recall
gs = GridSearchCV(
    KNeighborsClassifier(),
    grid_params,
    verbose = 1,
    cv = 4,
    n_jobs= -1,
    scoring=scoring,
    refit='deceased_recall'
    )

gs_results = gs.fit(X_train, y_train)
gs_results = gs.fit(X, y)

Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 14.6min finished


Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 21.7min finished


In [17]:
gs_results.best_score_

0.02917233711363131

In [18]:
gs_results.best_estimator_

KNeighborsClassifier(metric='euclidean', n_neighbors=38, weights='distance')

In [19]:
gs_results.best_params_

{'metric': 'euclidean', 'n_neighbors': 38, 'weights': 'distance'}

In [20]:
gs_results.cv_results_

{'mean_fit_time': array([64.58770877, 60.2518034 , 59.24603933, 51.14301759, 50.15597928,
        56.9834137 , 50.97933781, 51.89557314, 53.83034408, 54.06148022,
        57.16432977, 65.94005066, 59.06041199, 57.58617723, 57.99922138,
        53.23973244, 52.86612064, 56.48976964, 52.7183131 , 52.5220204 ,
        54.38946033, 49.96923524, 49.87065458, 51.14834929]),
 'std_fit_time': array([ 8.18199606, 14.7865112 ,  7.86131595, 11.68970707, 15.69562905,
        10.35993335, 12.04552228, 12.75817545, 10.15747786, 13.22553548,
        16.26622019, 11.64051829, 12.18236146, 14.03997585,  8.64195733,
        11.22297248, 14.08947219,  9.29974242, 11.42180216, 13.2934202 ,
         9.44331187, 11.47998926, 14.27770746,  3.10663458]),
 'mean_score_time': array([20.1757769 , 22.96391505, 24.03899449, 25.31346542, 24.4475736 ,
        26.79005581, 25.96347296, 24.53499389, 25.15102512, 27.05219209,
        26.30720055, 29.20148289, 28.32852328, 26.66580778, 25.02888513,
        25.28358138, 

In [21]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')

pd.DataFrame(gs_results.cv_results_).to_csv('out.zip', index=False,
          compression=compression_opts)